<a href="https://colab.research.google.com/github/adriroma07/pyspark-kafka-colab/blob/main/Pyspark_Kafka_1_0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install kafka in Google colab**

In [ ]:
!pip install kafka-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 246 kB 4.3 MB/s 


In [ ]:
!curl -sSOL https://downloads.apache.org/kafka/3.0.1/kafka_2.12-3.0.1.tgz

In [ ]:
!tar -xzf kafka_2.12-3.0.1.tgz

In [ ]:
!./kafka_2.12-3.0.1/bin/zookeeper-server-start.sh -daemon ./kafka_2.12-3.0.1/config/zookeeper.properties
!./kafka_2.12-3.0.1/bin/kafka-server-start.sh -daemon ./kafka_2.12-3.0.1/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


In [ ]:
!ps -ef | grep kafka

root         475       1 19 15:32 ?        00:00:02 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -XX:MaxInlineLevel=15 -Djava.awt.headless=true -Xlog:gc*:file=/content/kafka_2.12-3.0.1/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=100M -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/kafka_2.12-3.0.1/bin/../logs -Dlog4j.configuration=file:./kafka_2.12-3.0.1/bin/../config/log4j.properties -cp /content/kafka_2.12-3.0.1/bin/../libs/activation-1.1.1.jar:/content/kafka_2.12-3.0.1/bin/../libs/aopalliance-repackaged-2.6.1.jar:/content/kafka_2.12-3.0.1/bin/../libs/argparse4j-0.7.0.jar:/content/kafka_2.12-3.0.1/bin/../libs/audience-annotations-0.5.0.jar:/content/kafka_2.12-3.0.1/bin/../libs/commons-cli-1.4.jar:/content/kafka_2.12-3.0.1/bin/../libs/commons-lang3-3.8.1.jar:/content/kafka_2.12

### Create input and output topics.

The topics have the following configuration:

*   Replica = 1
*   Partition = 1

Se crean los diferentes topics:

*   **wikipedia-events** = wikipedia streaming data entry topic.
*   **outStreaming** = topic where the data is stored in kafka with the writerStream.
*   **groupData** = output topic for aggregation with pivoting.
*   **pivotData** = output topic for aggregation with grouping.



In [ ]:
!./kafka_2.12-3.0.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic wikipedia-events

Created topic wikipedia-events.


In [ ]:
!./kafka_2.12-3.0.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic outStreaming

Created topic outStreaming.


In [ ]:
!./kafka_2.12-3.0.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic groupData

Created topic groupData.


In [ ]:
!./kafka_2.12-3.0.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic pivotData

Created topic pivotData.


# **Download spark, install libraries and download dependencies**

To work in the environment, you have to update the system and download the version of java 8. Once installed, version 3.1.3 of spark is downloaded from the apache download page. In addition, pip installation of findspark, sseclient and pyspark is performed. Finally, we download different libraries that support spark.

In [ ]:
# --------- Spark -----------
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-3.1.3/spark-3.1.3-bin-hadoop2.7.tgz
!tar -xvf spark-3.1.3-bin-hadoop2.7.tgz
!pip install findspark
!pip install pyspark
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-10-assembly_2.12/3.1.3/spark-streaming-kafka-0-10-assembly_2.12-3.1.3.jar"
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.1.3/spark-sql-kafka-0-10_2.12-3.1.3.jar"

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [903 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main amd64 

### Imports and sets 

Modify the JAVA_HOME environment variable, add the SPARK_HOME variable and include the PYSPARK_SUBMIT_ARGS variable.

In [ ]:
from itertools import islice
import json
import time
import sys
import os
from datetime import datetime
import threading

In [ ]:
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.3-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/spark-sql-kafka-0-10_2.12-3.1.3.jar pyspark-shell'

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


# **Initialize findspark and create a sparkSession in Colab.**

In [ ]:
import findspark
findspark.init()

In [ ]:

import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
import time
import requests
from kafka import KafkaProducer
from json import dumps
import json

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext

In [ ]:
findspark.find()

'/content/spark-3.1.3-bin-hadoop2.7'

In [ ]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .config('spark.jars.packages','org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.3')\
        .getOrCreate()

# **Real-Time Data Extraction Wikipedia**

In the following section, the methods that make communication with https://stream.wikimedia.org/v2/stream/recentchange possible to extract the recent changes made by users in Wiki are developed.



**def init_namespaces():** Method that creates the first structure of the message.

In [ ]:
!pip install sseclient

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sseclient: filename=sseclient-0.0.27-py3-none-any.whl size=5584 sha256=538e5c1bfdfaf614a1ca567ee4c41ab1b4adcfc81bbd9c8fdc0e01907e66fc11
  Stored in directory: /root/.cache/pip/wheels/07/67/7e/96edf627ac746de1a5c5cbb8d59ed960f033b8352dc12c545d
Successfully built sseclient


In [ ]:
def init_namespaces():
    # create a dictionary for the various known namespaces
    # more info https://en.wikipedia.org/wiki/Wikipedia:Namespace#Programming
    namespace_dict = {-2: 'Media', 
                      -1: 'Special', 
                      0: 'main namespace', 
                      1: 'Talk', 
                      2: 'User', 3: 'User Talk',
                      4: 'Wikipedia', 5: 'Wikipedia Talk', 
                      6: 'File', 7: 'File Talk',
                      8: 'MediaWiki', 9: 'MediaWiki Talk', 
                      10: 'Template', 11: 'Template Talk', 
                      12: 'Help', 13: 'Help Talk', 
                      14: 'Category', 15: 'Category Talk', 
                      100: 'Portal', 101: 'Portal Talk',
                      108: 'Book', 109: 'Book Talk', 
                      118: 'Draft', 119: 'Draft Talk', 
                      446: 'Education Program', 447: 'Education Program Talk', 
                      710: 'TimedText', 711: 'TimedText Talk', 
                      828: 'Module', 829: 'Module Talk', 
                      2300: 'Gadget', 2301: 'Gadget Talk', 
                      2302: 'Gadget definition', 2303: 'Gadget definition Talk'}

    return namespace_dict

**def construct_event(event_data, user_types):** 
Method that defines the structure of the json event

In [ ]:
def construct_event(event_data, user_types):
    # use dictionary to change assign namespace value and catch any unknown namespaces (like ns 104)
    try:
        event_data['namespace'] = namespace_dict[event_data['namespace']]
    except KeyError:
        event_data['namespace'] = 'unknown'

    # assign user type value to either bot or human
    user_type = user_types[event_data['bot']]

    # define the structure of the json event that will be published to kafka topic
    event = {"id": event_data['id'],
             "domain": event_data['meta']['domain'],
             "namespace": event_data['namespace'],
             "title": event_data['title'],
             "timestamp": event_data['meta']['dt'],
             "user_name": event_data['user'],
             "user_type": user_type,
             "old_length": event_data['length']['old'],
             "new_length": event_data['length']['new']}
    
    return event

In the following code cell, the kafka producer is initialized, where the events will be inserted in json format. 
In addition, the topic is read in streaming and the WriteStream is used to save only the value field.

Run for a few seconds and stop running.

In [ ]:
producer = KafkaProducer(bootstrap_servers="localhost:9092", value_serializer=lambda x: dumps(x).encode('utf-8'),api_version=(0, 10, 1))

# used to parse user type
user_types = {True: 'bot', False: 'human'}
 
# init dictionary of namespaces
namespace_dict = init_namespaces()

import json
from sseclient import SSEClient as EventSource

url = 'https://stream.wikimedia.org/v2/stream/recentchange'
try:
  for event in EventSource(url):
      if event.event == 'message':
          try:
              change = json.loads(event.data)
              if change['type'] == 'edit':
                # construct valid json event
                event_to_send = construct_event(change, user_types)
                print("Event Streaming:",  event_to_send)   
                producer.send("wikipedia-events", value=event_to_send)

                dfWikiStream = spark \
                  .readStream \
                  .format("kafka") \
                  .option("kafka.bootstrap.servers", "localhost:9092") \
                  .option("subscribe", "wikipedia-events") \
                  .load()
                dfWikiFinalStream = dfWikiStream.selectExpr("CAST(value AS STRING)")

                ds = dfWikiStream \
                  .selectExpr("CAST(value AS STRING)") \
                  .writeStream \
                  .format("kafka") \
                  .option("kafka.bootstrap.servers", "localhost:9092") \
                  .option("topic", "outStreaming") \
                  .option("checkpointLocation", "/content/checkpoint") \
                  .start()
                  
          except ValueError:
              pass
except KeyboardInterrupt:
    print("process interrupted")

Event Streaming: {'id': 1753863194, 'domain': 'www.wikidata.org', 'namespace': 'main namespace', 'title': 'Q12499275', 'timestamp': '2022-08-10T15:37:22Z', 'user_name': 'Danil Satria', 'user_type': 'human', 'old_length': 22868, 'new_length': 23321}
Event Streaming: {'id': 150767170, 'domain': 'zh.wikipedia.org', 'namespace': 'main namespace', 'title': '中国大陆电视剧列表 (2022年)', 'timestamp': '2022-08-10T15:37:22Z', 'user_name': 'Musicinsummer', 'user_type': 'human', 'old_length': 217210, 'new_length': 217210}
Event Streaming: {'id': 1753863197, 'domain': 'www.wikidata.org', 'namespace': 'main namespace', 'title': 'Q12514428', 'timestamp': '2022-08-10T15:37:22Z', 'user_name': 'Danil Satria', 'user_type': 'human', 'old_length': 12211, 'new_length': 11797}
Event Streaming: {'id': 1753863196, 'domain': 'www.wikidata.org', 'namespace': 'main namespace', 'title': 'Q79330348', 'timestamp': '2022-08-10T15:37:22Z', 'user_name': 'Renamerr', 'user_type': 'human', 'old_length': 15911, 'new_length': 16044

In [ ]:
dfWiki = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "wikipedia-events") \
  .load()
dfWikiFinal = dfWiki.selectExpr("CAST(value AS STRING)")

# **Data transformation**

In [ ]:
display(dfWikiFinal.count())

39

In [ ]:
dfWikiFinal.show()

+--------------------+
|               value|
+--------------------+
|{"id": 1753863194...|
|{"id": 150767170,...|
|{"id": 1753863197...|
|{"id": 1753863196...|
|{"id": 1753863195...|
|{"id": 1753863198...|
|{"id": 1753863199...|
|{"id": 1753863200...|
|{"id": 1753863201...|
|{"id": 1753863202...|
|{"id": 1753863204...|
|{"id": 472769508,...|
|{"id": 1753863203...|
|{"id": 272415891,...|
|{"id": 1753863205...|
|{"id": 86214474, ...|
|{"id": 1753863206...|
|{"id": 1753863207...|
|{"id": 1753863208...|
|{"id": 1989627639...|
+--------------------+
only showing top 20 rows



In [ ]:
 from pyspark.sql.types import MapType, StringType 
 from pyspark.sql.functions import from_json
 dfWikiJson = dfWikiFinal.withColumn("value", from_json(dfWikiFinal.value, MapType(StringType(), StringType())))

In [ ]:
dfWikiJson.show()

+--------------------+
|               value|
+--------------------+
|{id -> 1753863194...|
|{id -> 150767170,...|
|{id -> 1753863197...|
|{id -> 1753863196...|
|{id -> 1753863195...|
|{id -> 1753863198...|
|{id -> 1753863199...|
|{id -> 1753863200...|
|{id -> 1753863201...|
|{id -> 1753863202...|
|{id -> 1753863204...|
|{id -> 472769508,...|
|{id -> 1753863203...|
|{id -> 272415891,...|
|{id -> 1753863205...|
|{id -> 86214474, ...|
|{id -> 1753863206...|
|{id -> 1753863207...|
|{id -> 1753863208...|
|{id -> 1989627639...|
+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import explode
df2 = dfWikiJson.select(dfWikiJson.value.alias("data"),explode(dfWikiJson.value))

In [ ]:
df2.show()

+--------------------+----------+---------------------------+
|                data|       key|                      value|
+--------------------+----------+---------------------------+
|{id -> 1753863194...|        id|                 1753863194|
|{id -> 1753863194...|    domain|           www.wikidata.org|
|{id -> 1753863194...| namespace|             main namespace|
|{id -> 1753863194...|     title|                  Q12499275|
|{id -> 1753863194...| timestamp|       2022-08-10T15:37:22Z|
|{id -> 1753863194...| user_name|               Danil Satria|
|{id -> 1753863194...| user_type|                      human|
|{id -> 1753863194...|old_length|                      22868|
|{id -> 1753863194...|new_length|                      23321|
|{id -> 150767170,...|        id|                  150767170|
|{id -> 150767170,...|    domain|           zh.wikipedia.org|
|{id -> 150767170,...| namespace|             main namespace|
|{id -> 150767170,...|     title|中国大陆电视剧列表 (2022年)|
|{id -> 150767170,

In [ ]:
df3 = df2.select(df2.data.cast("string"), df2.key, df2.value)

In [ ]:
df3.show()

+--------------------+----------+---------------------------+
|                data|       key|                      value|
+--------------------+----------+---------------------------+
|{id -> 1753863194...|        id|                 1753863194|
|{id -> 1753863194...|    domain|           www.wikidata.org|
|{id -> 1753863194...| namespace|             main namespace|
|{id -> 1753863194...|     title|                  Q12499275|
|{id -> 1753863194...| timestamp|       2022-08-10T15:37:22Z|
|{id -> 1753863194...| user_name|               Danil Satria|
|{id -> 1753863194...| user_type|                      human|
|{id -> 1753863194...|old_length|                      22868|
|{id -> 1753863194...|new_length|                      23321|
|{id -> 150767170,...|        id|                  150767170|
|{id -> 150767170,...|    domain|           zh.wikipedia.org|
|{id -> 150767170,...| namespace|             main namespace|
|{id -> 150767170,...|     title|中国大陆电视剧列表 (2022年)|
|{id -> 150767170,

In [ ]:
from pyspark.sql.functions import first

df4 = df3.groupby(df3.data).pivot("key").agg(first("value")).drop(df3.data)

In [ ]:
df4.show()                                                                           

+--------------------+----------+--------------+----------+----------+--------------------+---------------------------+------------------+---------+
|              domain|        id|     namespace|new_length|old_length|           timestamp|                      title|         user_name|user_type|
+--------------------+----------+--------------+----------+----------+--------------------+---------------------------+------------------+---------+
|commons.wikimedia...|1989627639|          File|      4121|      3685|2022-08-10T15:37:23Z|       File:Schloß Neuha...|    BotMultichillT|      bot|
|    www.wikidata.org|1753863203|main namespace|     28583|     35112|2022-08-10T15:37:19Z|                    Q374807|          HarryNº2|    human|
|    www.wikidata.org|1753863212|main namespace|      3521|      2322|2022-08-10T15:37:23Z|                  Q10474681|               SR5|    human|
|   arz.wikipedia.org|  39739901|main namespace|      3688|      3546|2022-08-10T15:37:22Z|       SHELS 01

## Aggregation with grouping

In [ ]:
dfGroupDomain = df4.groupby(df4.domain).count()

In [ ]:
dfGroupDomain.show()

+--------------------+-----+
|              domain|count|
+--------------------+-----+
|commons.wikimedia...|    4|
|    en.wikipedia.org|    1|
|    tr.wikipedia.org|    1|
|    es.wikipedia.org|    2|
|    no.wikipedia.org|    1|
|   arz.wikipedia.org|    1|
|    zh.wikipedia.org|    1|
|    www.wikidata.org|   26|
|    fr.wikipedia.org|    1|
|simple.wikipedia.org|    1|
+--------------------+-----+



In [ ]:
dfGroupDomainWrite = dfGroupDomain.select(create_map('domain', 'count').alias("value"))

In [ ]:
dfGroupDomainWrite.selectExpr("CAST(value AS STRING)") \
  .write \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("topic", "groupData") \
  .save()

In [ ]:
dfGroupDomainNamespace = df4.groupby(df4.domain, df4.namespace).count()

In [ ]:
dfGroupDomainNamespace.show()

+--------------------+--------------+-----+
|              domain|     namespace|count|
+--------------------+--------------+-----+
|    fr.wikipedia.org|main namespace|    1|
|    en.wikipedia.org|          Talk|    1|
|    www.wikidata.org|main namespace|   26|
|simple.wikipedia.org|main namespace|    1|
|    zh.wikipedia.org|main namespace|    1|
|    no.wikipedia.org|main namespace|    1|
|    es.wikipedia.org|main namespace|    2|
|commons.wikimedia...|          File|    4|
|    tr.wikipedia.org|main namespace|    1|
|   arz.wikipedia.org|main namespace|    1|
+--------------------+--------------+-----+



## Aggregation with Pivoting

In [ ]:
from pyspark.sql.functions import first

dfPivoting = df4.groupby(df4.id).pivot("user_type").agg(first("user_name"))

In [ ]:
dfPivoting.show()

+----------+------------------+--------------+
|        id|               bot|         human|
+----------+------------------+--------------+
|1753863203|              null|      HarryNº2|
|1753863205|              null|      Trilotat|
| 472769508|              null|      Pandamou|
|1753863211|              null|  Danil Satria|
|1753863219|              null|  Danil Satria|
|1753863201|              null|  Danil Satria|
|1753863200|              null|  Danil Satria|
|1753863197|              null|  Danil Satria|
|1753863196|              null|      Renamerr|
|1753863218|            Cewbot|          null|
|1989627640|     SchlurcherBot|          null|
| 272415891|              null|190.139.97.212|
|  18942085|              null|    MathXplore|
|1753863210|              null|       Romaine|
|1753863198|              null|          Yiyi|
|1753863206|              null|  Danil Satria|
|1753863209|            Cewbot|          null|
|1753863212|              null|           SR5|
|1989627639| 

## Aggregation with rollups and cubes

### Applying the rollup()

In [ ]:
df4.rollup(df4.user_type).count().show()

+---------+-----+
|user_type|count|
+---------+-----+
|     null|   39|
|    human|   30|
|      bot|    9|
+---------+-----+



In [ ]:
df4.rollup(df4.user_type, df4.namespace).count().show()

+---------+--------------+-----+
|user_type|     namespace|count|
+---------+--------------+-----+
|    human|          Talk|    1|
|    human|          null|   30|
|      bot|          File|    3|
|    human|main namespace|   28|
|    human|          File|    1|
|     null|          null|   39|
|      bot|main namespace|    6|
|      bot|          null|    9|
+---------+--------------+-----+



### Applying the cube()

In [ ]:
df4.cube(df4.user_type).count().show()

+---------+-----+
|user_type|count|
+---------+-----+
|     null|   39|
|    human|   30|
|      bot|    9|
+---------+-----+



In [ ]:
df4.cube(df4.user_type, df4.namespace).count().show()

+---------+--------------+-----+
|user_type|     namespace|count|
+---------+--------------+-----+
|    human|          Talk|    1|
|    human|          null|   30|
|      bot|          File|    3|
|     null|          Talk|    1|
|     null|main namespace|   34|
|    human|main namespace|   28|
|    human|          File|    1|
|     null|          null|   39|
|      bot|main namespace|    6|
|     null|          File|    4|
|      bot|          null|    9|
+---------+--------------+-----+



## Ranking functions

In [ ]:
from pyspark.sql.window import Window

windowPartition = Window.partitionBy("user_name").orderBy("new_length")

### Applying the row_number() function

In [ ]:
dfRowNumber = df4.withColumn("row_number",row_number().over(windowPartition))

In [ ]:
dfRowNumber.show()

+--------------------+----------+--------------+----------+----------+--------------------+---------------------------+------------------+---------+----------+
|              domain|        id|     namespace|new_length|old_length|           timestamp|                      title|         user_name|user_type|row_number|
+--------------------+----------+--------------+----------+----------+--------------------+---------------------------+------------------+---------+----------+
|    www.wikidata.org|1753863203|main namespace|     28583|     35112|2022-08-10T15:37:19Z|                    Q374807|          HarryNº2|    human|         1|
|    en.wikipedia.org|1537382771|          Talk|      4058|      3419|2022-08-10T15:37:22Z|       Talk:Pan-American...|Community Tech bot|    human|         1|
|commons.wikimedia...|1989627639|          File|      4121|      3685|2022-08-10T15:37:23Z|       File:Schloß Neuha...|    BotMultichillT|      bot|         1|
|    www.wikidata.org|1753863205|main na

In [ ]:
dfRowNumber.filter(dfRowNumber.row_number == 1).show()

+--------------------+----------+--------------+----------+----------+--------------------+---------------------------+------------------+---------+----------+
|              domain|        id|     namespace|new_length|old_length|           timestamp|                      title|         user_name|user_type|row_number|
+--------------------+----------+--------------+----------+----------+--------------------+---------------------------+------------------+---------+----------+
|    www.wikidata.org|1753863203|main namespace|     28583|     35112|2022-08-10T15:37:19Z|                    Q374807|          HarryNº2|    human|         1|
|    en.wikipedia.org|1537382771|          Talk|      4058|      3419|2022-08-10T15:37:22Z|       Talk:Pan-American...|Community Tech bot|    human|         1|
|commons.wikimedia...|1989627639|          File|      4121|      3685|2022-08-10T15:37:23Z|       File:Schloß Neuha...|    BotMultichillT|      bot|         1|
|    www.wikidata.org|1753863205|main na

### Applying the rank() function

In [ ]:
from pyspark.sql.functions import rank
 
df4.withColumn("rank", rank().over(windowPartition)).show()

+--------------------+----------+--------------+----------+----------+--------------------+---------------------------+------------------+---------+----+
|              domain|        id|     namespace|new_length|old_length|           timestamp|                      title|         user_name|user_type|rank|
+--------------------+----------+--------------+----------+----------+--------------------+---------------------------+------------------+---------+----+
|    www.wikidata.org|1753863203|main namespace|     28583|     35112|2022-08-10T15:37:19Z|                    Q374807|          HarryNº2|    human|   1|
|    en.wikipedia.org|1537382771|          Talk|      4058|      3419|2022-08-10T15:37:22Z|       Talk:Pan-American...|Community Tech bot|    human|   1|
|commons.wikimedia...|1989627639|          File|      4121|      3685|2022-08-10T15:37:23Z|       File:Schloß Neuha...|    BotMultichillT|      bot|   1|
|    www.wikidata.org|1753863205|main namespace|      5190|      5204|2022-0

### Applying the percent_rank() function

In [ ]:
from pyspark.sql.functions import percent_rank

df4.withColumn("percent_rank",percent_rank().over(windowPartition)).show()

+--------------------+----------+--------------+----------+----------+--------------------+---------------------------+------------------+---------+------------+
|              domain|        id|     namespace|new_length|old_length|           timestamp|                      title|         user_name|user_type|percent_rank|
+--------------------+----------+--------------+----------+----------+--------------------+---------------------------+------------------+---------+------------+
|    www.wikidata.org|1753863203|main namespace|     28583|     35112|2022-08-10T15:37:19Z|                    Q374807|          HarryNº2|    human|         0.0|
|    en.wikipedia.org|1537382771|          Talk|      4058|      3419|2022-08-10T15:37:22Z|       Talk:Pan-American...|Community Tech bot|    human|         0.0|
|commons.wikimedia...|1989627639|          File|      4121|      3685|2022-08-10T15:37:23Z|       File:Schloß Neuha...|    BotMultichillT|      bot|         0.0|
|    www.wikidata.org|175386

### Applying the dense_rank() function

In [ ]:
from pyspark.sql.functions import dense_rank
 
df4.withColumn("dense_rank",dense_rank().over(windowPartition)).show()

+--------------------+----------+--------------+----------+----------+--------------------+---------------------------+------------------+---------+----------+
|              domain|        id|     namespace|new_length|old_length|           timestamp|                      title|         user_name|user_type|dense_rank|
+--------------------+----------+--------------+----------+----------+--------------------+---------------------------+------------------+---------+----------+
|    www.wikidata.org|1753863203|main namespace|     28583|     35112|2022-08-10T15:37:19Z|                    Q374807|          HarryNº2|    human|         1|
|    en.wikipedia.org|1537382771|          Talk|      4058|      3419|2022-08-10T15:37:22Z|       Talk:Pan-American...|Community Tech bot|    human|         1|
|commons.wikimedia...|1989627639|          File|      4121|      3685|2022-08-10T15:37:23Z|       File:Schloß Neuha...|    BotMultichillT|      bot|         1|
|    www.wikidata.org|1753863205|main na

## Analytic functions

### Applying the cume_dist() functions

In [ ]:
from pyspark.sql.functions import cume_dist
 
df4.withColumn("cume_dist",cume_dist().over(windowPartition)).show()

+--------------------+----------+--------------+----------+----------+--------------------+---------------------------+------------------+---------+---------+
|              domain|        id|     namespace|new_length|old_length|           timestamp|                      title|         user_name|user_type|cume_dist|
+--------------------+----------+--------------+----------+----------+--------------------+---------------------------+------------------+---------+---------+
|    www.wikidata.org|1753863203|main namespace|     28583|     35112|2022-08-10T15:37:19Z|                    Q374807|          HarryNº2|    human|      1.0|
|    en.wikipedia.org|1537382771|          Talk|      4058|      3419|2022-08-10T15:37:22Z|       Talk:Pan-American...|Community Tech bot|    human|      1.0|
|commons.wikimedia...|1989627639|          File|      4121|      3685|2022-08-10T15:37:23Z|       File:Schloß Neuha...|    BotMultichillT|      bot|      1.0|
|    www.wikidata.org|1753863205|main namespac

### Applying the lag() functions

In [ ]:
from pyspark.sql.functions import lag
 
df4.withColumn("Lag", lag("old_length",2).over(windowPartition)).show()

+--------------------+----------+--------------+----------+----------+--------------------+---------------------------+------------------+---------+-----+
|              domain|        id|     namespace|new_length|old_length|           timestamp|                      title|         user_name|user_type|  Lag|
+--------------------+----------+--------------+----------+----------+--------------------+---------------------------+------------------+---------+-----+
|    www.wikidata.org|1753863203|main namespace|     28583|     35112|2022-08-10T15:37:19Z|                    Q374807|          HarryNº2|    human| null|
|    en.wikipedia.org|1537382771|          Talk|      4058|      3419|2022-08-10T15:37:22Z|       Talk:Pan-American...|Community Tech bot|    human| null|
|commons.wikimedia...|1989627639|          File|      4121|      3685|2022-08-10T15:37:23Z|       File:Schloß Neuha...|    BotMultichillT|      bot| null|
|    www.wikidata.org|1753863205|main namespace|      5190|      5204|

### Applying the lead() functions

In [ ]:
from pyspark.sql.functions import lead
 
df4.withColumn("Lead", lead("old_length", 2).over(windowPartition)).show()

+--------------------+----------+--------------+----------+----------+--------------------+---------------------------+------------------+---------+-----+
|              domain|        id|     namespace|new_length|old_length|           timestamp|                      title|         user_name|user_type| Lead|
+--------------------+----------+--------------+----------+----------+--------------------+---------------------------+------------------+---------+-----+
|    www.wikidata.org|1753863203|main namespace|     28583|     35112|2022-08-10T15:37:19Z|                    Q374807|          HarryNº2|    human| null|
|    en.wikipedia.org|1537382771|          Talk|      4058|      3419|2022-08-10T15:37:22Z|       Talk:Pan-American...|Community Tech bot|    human| null|
|commons.wikimedia...|1989627639|          File|      4121|      3685|2022-08-10T15:37:23Z|       File:Schloß Neuha...|    BotMultichillT|      bot| null|
|    www.wikidata.org|1753863205|main namespace|      5190|      5204|